In [8]:
#Calculation of cd index for the associahedra and 2-associahedra with SageMath code


#1.Implementation of the flag f vector using memorization

from functools import lru_cache

@lru_cache(maxsize=None)



def get_smaller_elements(start_elt, s = 1):
    #Given an start element the function returns all the elements that are s smaller then the start element.
    Nodes = [{} for i in range(s+1)]
    Q = Queue()
    Q.put(start_elt)
    #stores for each level the smaller elements
    Nodes[0][str(start_elt)] = start_elt
    while not Q.empty():
        curnode = Q.get()
        diff = curnode.codim - start_elt.codim
        if diff < s:
            for child in curnode.children:
                #check for every elements if it is yet in the Nodes list

                if not str(child) in Nodes[diff + 1]:
                    Nodes[diff + 1][str(child)] = child
                    Q.put(child)

    return Nodes[s]

def elt_flag_f_vector( S, elt):
    #given an element the function calculates all maximal chains startin with the original element
    #If S has length <= 1 there is only one chain consisting of only the element
    if len(S) <= 1:
        return 1
    diff = S[1] - S[0]
    total_sum = 0
    next_smaller = get_smaller_elements(elt, diff)
    for small in next_smaller:
        #recursively calculate the number of possibilities for chain starting with the smaller elements
        res = elt_flag_f_vector(S[1:], next_smaller[small])
        total_sum = total_sum + res

    return total_sum

def flag_f_vector(S, poset):
    #Calculates the flag f vector for a given Associahedra. Here viewd as a function
    #We reverse the order since the structure of the calculated poset only finds descending chains.
    S = list(reversed([poset.dim - s + 1 for s in S]))
    if not 0 in S:
        S = [0] + S
    return elt_flag_f_vector(S, poset.root)

def flag_h_vector(S, poset):
    #Calculates the flag h vector as defined
    res = 0
    subsets = [[subset for subset in Combinations(S, i)] for i in range(len(S))]
    for i in range(len(S)):
        for elt in subsets[i]:
            res = res + (-1) ** (len(S)-i) * flag_f_vector(list(elt), poset)
    return res




In [9]:
def omega(j):
    A.<c,d> = FreeAlgebra(QQ,2)
    if j % 2 == 0:
        return -1/2 *(c**2 - 2*d)**(int(j/2))
    else:
        return 1/2*c*(c**2 - 2*d)**(int((j-1)/2))

def cd_index(poset):
    #Implementation of the Stanleys formula for the CD-Index of Eulerian posets
    res = 0
    A.<c,d> = FreeAlgebra(QQ,2)
    N = list(np.arange(1, len(poset.nodes)))
    subsets = [[subset for subset in Combinations(N, i)] for i in range(len(N)+1)]
    subsets[0] = [[]]
    for i in range(1, len(N)+1):
        for S in subsets[i]:
            if S[0] % 2 == 1:
                prod = (c**2-2*d)**(int(1/2 * (S[0] - 1)))
                for i in range(1, len(S)):
                    prod = prod * omega(S[i] - S[i-1])
                prod = prod * omega(len(poset.nodes)-S[-1])
                prod = prod * flag_f_vector(list(S), poset)
                res = res + prod
    if (len(N)+1) % 2 == 1:
        res += (c**2-2*d)**(int(len(N)/2))
    return res


In [10]:
from Wn import *
from helper import *

W = Wn([2,0,0,0,0])
print(f'f_vector: {W.f_vector()}, alternating sum: {W.alternating()}')

f_vector: [42, 84, 56, 14, 1], alternating sum: 1


In [11]:
print(f'CD-index of W_{W.n} = {cd_index(W)}')

CD-index of W_[2, 0, 0, 0, 0] = 60*d^2 + 12*c^2*d + 42*c*d*c + 40*d*c^2 + c^4


In [12]:
from Kr import *

In [13]:
K = Kr(8)
print(K.f_vector())
print(f'CD-Index of K_{K.r} = {cd_index(K)}')

[429, 1287, 1485, 825, 225, 27, 1]
CD-Index of K_8 = 3598*d^3 + 775*c^2*d^2 + 2145*c*d*c*d + 2574*c*d^2*c + 1666*d*c^2*d + 3465*d*c*d*c + 2611*d^2*c^2 + 25*c^4*d + 198*c^3*d*c + 625*c^2*d*c^2 + 858*c*d*c^3 + 427*d*c^4 + c^6
